# Importing

In [21]:
import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Creating instance


In [22]:
all_params = cm_help.get_params()
COLLECTION_NAME = config.DISCOVERY_COLLECTION_NAME
print(COLLECTION_NAME)

Test


In [23]:
all_params['discovery']

{'default': 'a1',
 'r3': {'version': '2019-04-30',
  'apikey': 'e60nrAsq6WIQmWfuzv9_8by2yCXinHK_FOe3u9MSI1ml',
  'url': 'https://api.us-east.discovery.watson.cloud.ibm.com/instances/f2df1dc1-c923-49c1-80f3-7e23fc0441e8'},
 'a1': {'version': '2019-04-30',
  'apikey': 'h1IoNg4AIKbGtBs8n8QGf4dZ5tZwMpeVERQUrC1Q2-DF',
  'url': 'https://api.jp-tok.discovery.watson.cloud.ibm.com/instances/2330122b-7269-496b-ba7e-029f641470a1'},
 'version1': '2018-12-03',
 'apikey1': 'LTk2AGh0Mjsu3G9bGGSkU92xeCBAYdfa4lN4fjaA-CR4',
 'url1': 'https://gateway.watsonplatform.net/discovery/api'}

In [24]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [25]:
discovery_config = discovery_init(COLLECTION_NAME='Naviga1',default="r3")
#COLLECTION_NAME = 'Naviga1'
#discovery_config = dh_help.discovery_init(COLLECTION_NAME=COLLECTION_NAME,default="r3")

Discovery instance loaded r3
COLLECTION ID: ec586f27-f848-40b7-99af-dab80a43b248 COLLECTION NAME: Naviga1


# NULL query

In [27]:
offset = 0
count = 10
reqparams = {'offset':offset,'count':count}
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)

offset in query = 0


# Getting html text

In [31]:
offset = 0
count = 10
reqparams = {'offset':offset,'count':count}
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)
print("len "+str(len(discovery_data['results'])))
print(discovery_data['results'][0]['html'])

offset in query = 0
len 10

<p>2020 has been bizarre - you don't need me to tell you that.</p>
<p>Basketball has not been excluded from the oddities of a year that began with the death of NBA legend <person>Kobe Bryant</person>, followed by a finals push by <person>LeBron James</person> and the Lakers that was abruptly halted, culminating with "The Last Dance" documentary that detailed the greatness of <person>Michael Jordan</person> and the <org idsrc="xmltag.org" value="ACORN:6001131364">Chicago Bulls'</org> dynasty of the 1990s.</p>

<p>That's just what's happened through the first five months.</p>

<p>The discussion of who the greatest NBA player of all time is or who belongs on the <location>Mount Rushmore</location> of the sport became a reawakened topic after <org>ESPN's</org> 10-part series about the Bulls' sixth title in an eight year span, that concluded with the final two episodes on Sunday. It might be fair to say, with little distraction by the lack of live sports, that th

# REGEX

In [32]:
batRegex = re.compile(r'([a-zA-Z]{2}\s<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')
m1=batRegex.search(source)

In [34]:
print (re.findall(batRegex,source))

[('or <person>Lisa Holt</person>', 'Lisa Holt'), ('of <org>Main Street Barber Shop</org>', 'Main Street Barber Shop'), ('in <location idsrc="xmltag.org" value="LU/us.nm.clovis">Clovis</location>', 'Clovis'), ('he <org idsrc="xmltag.org" value="ACORN:0532342767">New Mexico Department of Workforce Solutions</org>', 'New Mexico Department of Workforce Solutions'), ('an <money>$2,000</money>', '$2,000'), ('he <org>Santa Fe Reporter</org>', 'Santa Fe Reporter'), ('ry <person>Bill McCamley</person>', 'Bill McCamley'), ('id <location idsrc="xmltag.org" value="LS/us.nm">New Mexico</location>', 'New Mexico'), ('ts <chron>April 22</chron>', 'April 22'), ('he <org idsrc="xmltag.org" value="ACORN:1576954704">U.S. Department of Labor</org>', 'U.S. Department of Labor')]


In [160]:
lists = re.findall(batRegex,source)

In [161]:
len_ = len(lists)

In [162]:
for i in range(len_):
    lists[i] = list(lists[i])

In [163]:
lists

[['or <person>Lisa Holt</person>', 'Lisa Holt'],
 ['of <org>Main Street Barber Shop</org>', 'Main Street Barber Shop'],
 ['in <location idsrc="xmltag.org" value="LU/us.nm.clovis">Clovis</location>',
  'Clovis'],
 ['he <org idsrc="xmltag.org" value="ACORN:0532342767">New Mexico Department of Workforce Solutions</org>',
  'New Mexico Department of Workforce Solutions'],
 ['an <money>$2,000</money>', '$2,000'],
 ['he <org>Santa Fe Reporter</org>', 'Santa Fe Reporter'],
 ['ry <person>Bill McCamley</person>', 'Bill McCamley'],
 ['id <location idsrc="xmltag.org" value="LS/us.nm">New Mexico</location>',
  'New Mexico'],
 ['ts <chron>April 22</chron>', 'April 22'],
 ['he <org idsrc="xmltag.org" value="ACORN:1576954704">U.S. Department of Labor</org>',
  'U.S. Department of Labor']]

In [164]:
regex = re.compile(r'\svalue="(.*?)"')
subst = ''
for i in range(len_):
    bregex = re.findall(regex,lists[i][0])
    if(len(bregex)!=0):
        lists[i][0] = lists[i][0].replace(bregex[0], subst)

regex = re.compile(r'\sidsrc="(.*?)"')
for i in range(len_):
    bregex = re.findall(regex,lists[i][0])
    if(len(bregex)!=0):
        lists[i][0] = lists[i][0].replace(bregex[0], subst)
    

In [165]:
lists

[['or <person>Lisa Holt</person>', 'Lisa Holt'],
 ['of <org>Main Street Barber Shop</org>', 'Main Street Barber Shop'],
 ['in <location idsrc="" value="">Clovis</location>', 'Clovis'],
 ['he <org idsrc="" value="">New Mexico Department of Workforce Solutions</org>',
  'New Mexico Department of Workforce Solutions'],
 ['an <money>$2,000</money>', '$2,000'],
 ['he <org>Santa Fe Reporter</org>', 'Santa Fe Reporter'],
 ['ry <person>Bill McCamley</person>', 'Bill McCamley'],
 ['id <location idsrc="" value="">New Mexico</location>', 'New Mexico'],
 ['ts <chron>April 22</chron>', 'April 22'],
 ['he <org idsrc="" value="">U.S. Department of Labor</org>',
  'U.S. Department of Labor']]

In [166]:
res = {}
for i in range(len_):
    lists[i][0]=lists[i][0].split('/')
    lists[i][0][1]=lists[i][0][1].split('>')
    keyword = lists[i][0][1][0]
    value = lists[i][1]
    
    if keyword in res:
        res[keyword].append(value)       
    else:
        res[keyword] = [value]

In [167]:
res

{'person': ['Lisa Holt', 'Bill McCamley'],
 'org': ['Main Street Barber Shop',
  'New Mexico Department of Workforce Solutions',
  'Santa Fe Reporter',
  'U.S. Department of Labor'],
 'location': ['Clovis', 'New Mexico'],
 'money': ['$2,000'],
 'chron': ['April 22']}

# Error handling

In [ ]:
discovery_data = dh_help.discovery_init(COLLECTION_NAME='Naviga1')

In [ ]:
import json
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../modules/")
import modules.common_helper as cm_help
from ibm_watson import DiscoveryV1
import modules.config as config
import modules.discovery_helper as dh
import os
import time
from pprint import pprint
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
all_params = cm_help.get_params()
COLLECTION_NAME = config.DISCOVERY_COLLECTION_NAME
print(COLLECTION_NAME)

In [ ]:
all_params['discovery']

In [ ]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [ ]:
discovery_data = discovery_init(COLLECTION_NAME='Naviga1',default="r3")

In [ ]:
dict_ = []
result = discovery_data['results']

for obj in result:
    temp={}
    temp['id'] = obj['id']

    # dict for 0th doc
    res = {}
    for tag in soup.findAll('p'):
        #tag.find(x for x in ent_types)
        for ent in ent_types:
            try:
                tag.find(ent)
                res[ent].append(tag.find(ent).text)
            except Exception as e:
                pass
    # for extra tags not in ent_types
    for tag in soup.findAll('chron'):
        res['chron'].append(tag.text)
    
    temp = {**temp,**res}
    dict_.append(temp)